In [49]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss
from torch.optim import Adam

from tqdm import tqdm

In [50]:
DEBUG = True
DEVICE = torch.device('cuda:0') if torch.cuda.is_available() and not DEBUG else 'cpu'

In [51]:
class CNNModel(nn.Module):
    def __init__(self, device='cpu'):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 1, 5, padding=(2, 1), padding_mode="reflect")
        self.conv2 = nn.Conv2d(1, 1, 5, padding=(2, 1), padding_mode="reflect")
        self.conv3 = nn.Conv2d(1, 1, 5, padding=(2, 1), padding_mode="reflect")
        self.conv4 = nn.Conv2d(1, 1, 5, padding=(2, 1), padding_mode="reflect")
        self.conv5 = nn.Conv2d(1, 1, 5, padding=(2, 1), padding_mode="reflect")
        self.pool = nn.MaxPool1d(1, 3)
        self.softmax = nn.Softmax(dim=1)
        self.to(device)


    def forward(self, x):
        x = self.conv1(x)
        # print(x.shape)
        x = self.conv2(x)
        # print(x.shape)
        x = self.conv3(x)
        # print(x.shape)
        x = self.conv4(x)
        # print(x.shape)
        x = self.conv5(x)
        # print(x.shape)
        x = self.pool(x)
        # print(x.shape)
        x = self.softmax(x)
        return x

In [52]:
import DataImport
INPUT_FILE = "train.csv"

x_train, y_train = DataImport.import_data(INPUT_FILE, DEVICE)
model = CNNModel()
print("input shape" + str(x_train.shape))
test_output = model(x_train)
print("output shape" + str(test_output.shape))
# print(test_output)

input shapetorch.Size([1, 70322, 13])
output shapetorch.Size([1, 70322, 1])


In [53]:
def train(model, x_train, y_train, n_epochs=5, lr=0.0001):
    optimizer = Adam(model.parameters(), lr=lr)
    criterion = nn.BCELoss()

    losses = []
    for epoch in tqdm(range(n_epochs)):
        optimizer.zero_grad()

        output_train = model(x_train)
        output_train = output_train.reshape((output_train.shape[1], output_train.shape[2]))
        # print(output_train)
        loss_train = criterion(output_train, y_train)
        loss_train.backward()
        optimizer.step()
        loss_value = loss_train.item()
        losses.append(loss_value)
        if epoch % 100 == 0:
            print("Epoch {}: loss = {}".format(str(epoch), str(loss_value)))

        if len(output_train.nonzero()) == 0:        # not sure if this works
            print("Epoch {}: all zero output!".format(str(epoch)))

    return losses

def calc_acc(y_pred, y, th=1.0):
    #tp = 0
    #tn = 0
    size = len(y)
    #for i in range(size):
    #    if y_pred[i] >= th and y[i] == 1:
    #        tp += 1
    #    elif y_pred[i] < th and y[i] == 0:
    #        tn += 1
    #return float(tp + tn)*100/size
    with torch.no_grad():
        #print(y)
        #print(y_pred)
        y_th = torch.heaviside(y_pred - torch.tensor(th).expand_as(y_pred), torch.tensor(1.0).expand_as(y_pred))
        tp = torch.sum(torch.logical_and(y_th, y))
        p_pred = torch.sum(y_th)
        p_label = torch.sum(y)
        #print(y_th)
        #print(torch.logical_xor(y_th, y))
        acc = float(size - torch.sum(torch.logical_xor(y_th, y)))*100/size
        prec = float(tp)*100/p_pred
        rec = float(tp)*100/p_label
    return [acc, prec, rec]

In [54]:
model = CNNModel(DEVICE)
losses = train(model, x_train, y_train, n_epochs=100, lr=0.001)
# print(losses)

  1%|          | 1/100 [00:00<00:35,  2.78it/s]

Epoch 0: loss = 51.798866271972656


100%|██████████| 100/100 [00:33<00:00,  2.97it/s]


In [55]:
with torch.no_grad():
    y_prediction = model(x_train)
    y_prediction = y_prediction.reshape((y_prediction.shape[1], y_prediction.shape[2]))
    evauluation_result = calc_acc(y_prediction, y_train)
    print(evauluation_result)

[48.201131935951764, tensor(0.), tensor(0.)]
